#### generate allele count script

In [1]:
### index vcf
# java -jar /home/cjr/software/gatk-4.1.2.0/gatk-package-4.1.2.0-local.jar IndexFeatureFile
# -F ind1_gatk_het_SNP.vcf

In [1]:
### 90 samples
ind_dict = {
    "ind1": ['brain_IP_1', 'heart_IP_1', 'liver_IP_1'],
    "ind2":
    ['brain_IP_2', 'heart_IP_2', 'placenta_IP_2', 'kidney_IP_2', 'liver_IP_2'],
    "ind3": ['brain_IP_3', 'heart_IP_3', 'liver_IP_3', 'kidney_IP_3'],
    "ind4": [
        'stomach_IP_4', 'muscle_IP_4', 'lung_IP_4', 'kidney_IP_4',
        'placenta_IP_4'
    ],
    "ind5": ['stomach_IP_5', 'muscle_IP_5', 'lung_IP_5'],
    "ind6": ['placenta_IP_6'],
    "ind7": [
        'Adipose-1-1', 'Adrenal_gland-1-1', 'Aorta-1-1', 'Heart-1-1',
        'Skin-1-1', 'Spleen-1-1'
    ],
    "ind8": [
        'Lung-2-1', 'Lung-2-4', 'Spleen-2-1', 'Tongue-2-1',
        'Urinary_bladder-2-1'
    ],
    "ind9":
    ['Appendix-3-2', 'Colon-3-2', 'Esophagus-3-2', 'Muscle-3-2', 'Spleen-3-2'],
    "ind10": [
        'Aorta-4-2', 'Esophagus-4-2', 'Heart-4-2', 'Jejunum-4-2', 'Liver-4-2',
        'Lung-4-2', 'Lung-4-4', 'Prostate-4-2', 'Rectum-4-2', 'Skin-4-2',
        'Stomach-4-2', 'Testis-4-2', 'Thyroid_gland-4-2', 'Urinary_bladder-4-2'
    ],
    "ind11": [
        'Appendix-5-3', 'Brainstem-5-3', 'Cerebellum-5-3', 'Cerebrum-5-3',
        'Duodenum-5-3', 'Hypothalamus-5-3', 'Jejunum-5-3', 'Muscle-5-3',
        'Rectum-5-3', 'Stomach-5-3', 'Thyroid_gland-5-3', 'Trachea-5-3',
        'Urinary_bladder-5-3'
    ],
    "ind12": ['Cerebrum-6-3'],
    "ind13": ['Cerebellum-7-4'],
    "ind14": [
        'FrontalCortex_1', 'Cerebellum_1', 'Heart_1', 'Liver_1', 'Lung_1',
        'Kidney_1', 'Spleen_1'
    ],
    "ind15": [
        'FrontalCortex_2', 'Cerebellum_2', 'Heart_2', 'Liver_2', 'Kidney_2',
        'Spleen_2', 'Muscle_1'
    ],
    "ind16": [
        'FrontalCortex_3', 'Cerebellum_3', 'Heart_3', 'Liver_3', 'Lung_2',
        'Muscle_2'
    ],
    "ind17": ['Muscle_3'],
    "ind18": ['Lung_3', 'Kidney_3', 'Spleen_3']
}

In [3]:
# os.chdir("/Charles/project/ASm6A/by_ind/count/")
os.chdir(
    "/home/galaxy/project/alleleSpecific_analysis/data/bam_wasp/corrected_count/"
)

vcf_dir = "/Charles/project/ASm6A/by_ind/SNP_calling/filtered_vcf/"
# input_bam_dir = "/Charles/project/ASm6A/corrected_input/"
# ip_bam_dir = "/Charles/project/ASm6A/corrected_ip/"
input_bam_dir = "/home/galaxy/project/alleleSpecific_analysis/data/bam_wasp/corrected_input/"
ip_bam_dir = "/home/galaxy/project/alleleSpecific_analysis/data/bam_wasp/corrected_ip/"

prefix = "python /home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/run_GATK_ASEReadCounter.py /usr/local/bin/GenomeAnalysisTK.jar /home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19.fa"
# postfix = "--disable_drf"


def alter_ip_name():
    map_file = "/home/galaxy/project/alleleSpecific_analysis/data/sample_info/map.txt"
    df = pd.read_table(map_file, header=None)
    map_dict = dict(zip(df.iloc[:, 1], df.iloc[:, 0]))
    return map_dict


def get_input_name():
    map_file = "/home/galaxy/project/alleleSpecific_analysis/data/sample_info/total_Input_vs_IP.txt"
    df = pd.read_table(map_file)
    sample_dict = dict(zip(df['IP'], df['Input']))
    return sample_dict


map_dict = alter_ip_name()
sample_dict = get_input_name()
with open("run.sh", 'w') as fw:
    for ind, ip_list in ind_dict.items():
        vcf = os.path.join(vcf_dir, "%s_gatk_het_SNP.vcf" % ind)
        for ip_sample in ip_list:
            input_sample, ip_bam = "", ""
            ip_bam = os.path.join(ip_bam_dir,
                                  "%s/%s-uniq.bam" % (ip_sample, ip_sample))
            if not os.path.exists(ip_bam):
                ip_bam = os.path.join(
                    ip_bam_dir, "%s/%s-uniq.bam" %
                    (map_dict[ip_sample], map_dict[ip_sample]))
            if ip_sample in sample_dict:
                input_sample = sample_dict[ip_sample]
            else:
                input_sample = sample_dict[map_dict[ip_sample]]
            input_bam = os.path.join(
                input_bam_dir, "%s/%s-uniq.bam" % (input_sample, input_sample))
            #
            ip_result_dir = "ip_count/%s" % os.path.basename(ip_bam).split(
                "-uniq.bam")[0]
            input_result_dir = "input_count/%s" % os.path.basename(
                input_bam).split("-uniq.bam")[0]
            #
            fw.write("%s %s %s %s\n" %
                     (prefix, vcf, ip_bam, ip_result_dir))  # , postfix
            fw.write("%s %s %s %s\n" %
                     (prefix, vcf, input_bam, input_result_dir))  # , postfix

In [ ]:
#### nohup less run.sh | parallel -j20 &